<a href="https://colab.research.google.com/github/subham1/sentence-transformers/blob/master/QuoraSentenceSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sentence_transformers 

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 450kB 9.7MB/s 
     |████████████████████████████████| 1.0MB 65.3MB/s 
     |████████████████████████████████| 870kB 55.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5-cp36-none-any.whl size=64942 sha256=82751db0e823d04e74b4c69ad153b22785840e55e230eeeea3453ff4731a2236
  Stored in directory: /root/.cache/pip/wheels/b4/ce/39/5bbda8ac34eb52df8c6531382ca077773fbfcbfb6386e5d66c
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=ade71cfeb41035dc4495253454fff104f3ad8550e560aa4f5363d4af53ddfb60
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [0]:
ls

In [15]:
cd '/content/drive/My Drive/sbert/sentence-transformers'

/content/drive/My Drive/sbert/sentence-transformers


In [7]:
ls

docs/      NOTICE.txt                     requirements.txt        setup.py
examples/  QuoraSentenceSimilarity.ipynb  sentence_transformers/  train.csv
LICENSE    README.md                      setup.cfg


In [8]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import scipy.spatial
from torch.utils.data import DataLoader

In [0]:
df =pd.read_csv('/train.csv', header=None)

In [0]:
df[3] = df[3].astype(str)
df[4] = df[4].astype(str)

In [44]:
df.head()

,0,1,2,3,4,5
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
# from sentence_transformers.readers.QuoraDataReader import QuoraDataReader
import logging
from datetime import datetime

In [0]:
from sentence_transformers.readers import InputExample
import csv
import gzip
import os
import pandas as pd
class QuoraDataReader:
    """
    Reads in the STS dataset. Each line contains two sentences (s1_col_idx, s2_col_idx) and one label (score_col_idx)
    """
    def __init__(self, dataset_folder, s1_col_idx=3, s2_col_idx=4, score_col_idx=5, delimiter="\t",
                 quoting=csv.QUOTE_NONE, normalize_scores=True, min_score=0, max_score=1):
        self.dataset_folder = dataset_folder
        self.score_col_idx = score_col_idx
        self.s1_col_idx = s1_col_idx
        self.s2_col_idx = s2_col_idx
        self.delimiter = delimiter
        self.quoting = quoting
        self.normalize_scores = normalize_scores
        self.min_score = min_score
        self.max_score = max_score

    def get_examples(self, filename, max_examples=0):
        """
        filename specified which data split to use (train.csv, dev.csv, test.csv).
        """
        data = csv.reader(open(os.path.join(self.dataset_folder, filename), encoding="utf-8"),
                          delimiter=self.delimiter, quoting=self.quoting)
        df =pd.read_csv(os.path.join(self.dataset_folder, filename), header =None)
        df[self.s1_col_idx] = df[self.s1_col_idx].astype(str)
        df[self.s2_col_idx] = df[self.s2_col_idx].astype(str)
        examples = []
        
        for id,row in df.iterrows():
            score =int(row[self.score_col_idx])
            if self.normalize_scores:  # Normalize to a 0...1 value
                score = (score - self.min_score) / (self.max_score - self.min_score)

            s1 = row[self.s1_col_idx]
            s2 = row[self.s2_col_idx]
            examples.append(InputExample(guid=filename+str(id), texts=[s1, s2], label=score))

            if max_examples > 0 and len(examples) >= max_examples:
                break

        return examples

In [19]:
pwd

'/content/drive/My Drive/sbert/sentence-transformers'

In [25]:
ls '/content'

drive/           NOTICE.txt          README.md         setup.cfg
InputExample.py  __pycache__/        requirements.txt  setup.py
LICENSE          QuoraDataReader.py  sample_data/


In [0]:
quora_reader = QuoraDataReader('/') 

In [0]:
model_name = 'bert-base-nli-mean-tokens'
train_batch_size = 16
num_epochs = 4
model_save_path = 'sentence-transformers-master/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [32]:

model = SentenceTransformer(model_name)

100%|██████████| 405M/405M [00:31<00:00, 13.0MB/s]


In [0]:
train_data = SentencesDataset(quora_reader.get_examples('train.csv'), model)


In [0]:
tmp1 =df[1].tolist()
tmp2 =df[2].tolist()
k =tmp1 + tmp2

In [0]:
tmp1 =df[3].tolist()
tmp2 =df[4].tolist()
v =tmp1 + tmp2

In [0]:
from collections import OrderedDict 
res = OrderedDict(zip(k, v)) 

In [0]:
corpus = list(res.values())

In [49]:
len(corpus)

537933

In [0]:
embeddings =model.encode(corpus)